In [11]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import os

In [4]:
load_dotenv()

True

In [6]:
llm = HuggingFaceEndpoint(
    repo_id= 'meta-llama/Llama-3.2-3B-Instruct',
    task = 'text-generation',
    huggingfacehub_api_token = os.getenv('HUGGINGFACEHUB_ACCESS_TOKEN')
)

In [7]:
model = ChatHuggingFace(llm=llm, temperature=0)

In [8]:
class LLMState(TypedDict):

    question: str
    answer: str

In [9]:
def llm_qa(state: LLMState) -> LLMState:
    
    question = state['question']
    answer = model.invoke(question).content
    state['answer'] = answer
    return state


In [13]:
graph = StateGraph(LLMState)

graph.add_node('llm_qa', llm_qa)

graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

workflow = graph.compile()

In [19]:
input_state = {'question': 'Tell me a joke.'}

final_state = workflow.invoke(input_state)

print(final_state['answer'])

A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?"

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."
